In [14]:
import pandas as pd

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA


import tkinter as tk
from tkinter import scrolledtext
from langchain_ollama import OllamaLLM


In [2]:
# Charger les fichiers CSV
data_1 = pd.read_csv('/home/cytech/test/Rag_covid/data/data_preprocessing/Age_Preprocessing.csv')
data_2 = pd.read_csv('/home/cytech/test/Rag_covid/data/data_preprocessing/Diabetes_Preprocessing.csv')
data_3 = pd.read_csv('/home/cytech/test/Rag_covid/data/data_preprocessing/Overweight_or_obese_Preprocessing.csv')

# Concaténer les datasets si nécessaire
dataset = pd.concat([data_1, data_2, data_3], ignore_index=True)


In [10]:
# Remplacer les valeurs manquantes par des chaînes vides et convertir en type str
dataset['context'] = dataset['context'].fillna('').astype(str)


In [11]:
# Supposons que la colonne 'context' contient le texte pertinent
documents = dataset['context'].tolist()


In [12]:
# Initialiser le modèle d'embedding
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Créer l'index Chroma
vectorstore = Chroma.from_texts(documents, embedding_model)


In [17]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [16]:
llm = OllamaLLM(model="llama3.2:3b")


In [18]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)


In [ ]:
query = "Quelle est l'efficacité du vaccin contre le COVID-19 ?"
response = qa_chain.run(query)
print(response)


/tmp/ipykernel_537259/1745571089.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Je n'ai pas trouvé d'étude ou de données qui mentionnent l'efficacité du vaccin contre le COVID-19 dans ces sources. Les études présentées sont toutes des observations retrospectives sur les patients hospitalisés pour COVID-19, mais elles ne traitent pas explicitement la question de l'efficacité des vaccinations contre cette maladie.

Si vous recherchez des informations sur l'efficacité du vaccin contre le COVID-19, je vous recommande de consulter les sources les plus récentes et les plus fiables, telles que les publications scientifiques dans des revues médicales ou les rapports officiels de santé publique.


In [20]:
query = "Give me figures on covid and overweight"
response = qa_chain.run(query)
print(response)


Based on the provided studies, here are some figures related to COVID-19 and being overweight:

1. Obesity as a risk factor for Covid-19 hospital admission:
	* Age < 60 years: OR 1.8 (Severe), with a significant p-value (<0.01)
	* Age > 60 years: OR 1.1 (not significant, with a p-value of 0.57)
2. Obesity and COVID-19 mortality risk:
	* Age < 60 years: Fatality OR not calculated
	* Age > 60 years: Fatality OR 2.06 (not significant, with a p-value of 0.55), but fatality range is quite broad (0.19-22.44) and the confidence interval suggests that there may be an association, although it's not statistically significant at current levels.

It's worth noting that these studies are observational in nature, so while they provide some insight into the relationship between obesity and COVID-19 outcomes, they cannot establish causality. Additionally, the sample sizes are relatively small compared to larger studies or meta-analyses.

If you're looking for more information or want to explore furthe

In [15]:
chat_history = []

def get_ai_response():
    """Récupère la réponse du modèle et met à jour l'historique."""
    user_input = entry.get().strip()
    if not user_input:
        output_text.insert(tk.END, "Veuillez entrer un message.\n")
        return

    chat_history.append({"role": "user", "content": user_input})

    formatted_prompt = "\n".join([f"{msg['role']}: {msg['content']}" for msg in chat_history])

    response = llm.invoke(formatted_prompt)

    chat_history.append({"role": "assistant", "content": response})

    output_text.insert(tk.END, f"Utilisateur : {user_input}\nIA : {response}\n\n")

    entry.delete(0, tk.END)

In [ ]:
# Configuration de la fenêtre principale
root = tk.Tk()
root.title("Chat avec Llama3")
root.geometry("600x400")

# Champ de saisie
entry = tk.Entry(root, width=60)
entry.pack(pady=10)

# Bouton d'envoi
send_button = tk.Button(root, text="Envoyer", command=get_ai_response)
send_button.pack()

# Zone de texte pour afficher la conversation
output_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=70, height=15)
output_text.pack(pady=10)

# Lancement de la boucle principale de l'interface
root.mainloop()